In [ ]:
!pip install torchxrayvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import tqdm
import sklearn, sklearn.metrics
import pandas as pd
import torchxrayvision as xrv
from google.colab import drive
import skimage
from PIL import Image

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datapath = '/content/drive/My Drive/data/NIH/'
# import os
# os.chdir('/content/drive/My Drive/data/NIH/')

In [ ]:
ls

20-img/              Effusion.csv         images-224/       Untitled0.ipynb
BBox_List_2017.csv   example_based.ipynb  Model/
Data_Entry_2017.csv  example_based.py     new_Effusion.csv


In [ ]:
img_path = '20-img/'
label_path = 'Effusion.csv'
new_label_path = 'new_Effusion.csv'

In [ ]:
import pandas as pd
import os

# Path to directory containing images
image_dir = datapath+img_path

# List image names in directory
image_names = os.listdir(image_dir)

# Path to CSV file
csv_file = datapath+label_path

# Read CSV into Pandas DataFrame
df = pd.read_csv(csv_file)

# Filter DataFrame to only include rows with image names in image_names list
filtered_df = df[df['Image Index'].isin(image_names)]

# Write filtered DataFrame to new CSV file
filtered_csv_file = datapath+new_label_path
filtered_df.to_csv(filtered_csv_file, index=False)
filtered_df

,Unnamed: 0,Image Index,label
22,22,00000007_000.png,0
46,46,00000013_008.png,0
47,47,00000013_009.png,0
58,58,00000013_020.png,0
59,59,00000013_021.png,1
61,61,00000013_023.png,0
72,72,00000013_034.png,0
73,73,00000013_035.png,0
75,75,00000013_037.png,0
88,88,00000017_000.png,1


In [ ]:
filtered_df = filtered_df.drop(columns='Unnamed: 0')
filtered_df = filtered_df.reset_index(drop=True)
filtered_df

,Image Index,label
0,00000007_000.png,0
1,00000013_008.png,0
2,00000013_009.png,0
3,00000013_020.png,0
4,00000013_021.png,1
5,00000013_023.png,0
6,00000013_034.png,0
7,00000013_035.png,0
8,00000013_037.png,0
9,00000017_000.png,1


In [ ]:
filtered_csv_file = datapath+new_label_path
filtered_df.to_csv(filtered_csv_file, index=False)

In [ ]:
import pandas as pd
import cv2
import numpy as np
import pickle

l_path = datapath + new_label_path
i_path = datapath + img_path

def create_pickle_file(images_path, labels_path, pickle_path_images, pickle_path_labels, pickle_path_names):
    
    # load data
    labels_df = pd.read_csv(labels_path)
    image_names = labels_df['Image Index'].values
    labels = labels_df['label'].values
    
    # create empty lists
    images = []
    path_names = []
    
    # loop through each image and label
    for i, image_name in enumerate(image_names):
        # read image
        image_path = images_path + image_name
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        print(image.shape)
        
        # preprocess image
        # image = pre_process_image(image)
        
        # append image and label to lists
        images.append(image)
        path_names.append(image_path)
        
        # print progress
        if i % 500 == 0:
            print(f"{i} images processed.")
    
    # convert lists to numpy arrays
    images = np.array(images)
    path_names = np.array(path_names)
    
    # save images, labels, and path names as pickle files
    with open(pickle_path_images, "wb") as f:
        pickle.dump(images, f)
    
    with open(pickle_path_labels, "wb") as f:
        pickle.dump(labels, f)
        
    with open(pickle_path_names, "wb") as f:
        pickle.dump(path_names, f)
    
    print("Pickle files created successfully.")


In [ ]:
create_pickle_file(i_path, l_path, datapath+"validation_pleural_images_.pickle", datapath+"validation_pleural_labels_.pickle", datapath+"validation_pleural_path_names_.pickle")

(224, 224)
0 images processed.
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
Pickle files created successfully.


In [ ]:
import os
import random
import shutil

folder_path = i_path  # Replace with the path to your folder
num_images = 30  # Replace with the number of images you want to select
output_folder_path = datapath+"30-img"  # Replace with the path to your output folder

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Get a list of all the image filenames in the folder
image_filenames = [f for f in os.listdir(folder_path) if f.endswith(".jpg") or f.endswith(".png")]

# Randomly select the desired number of images
selected_images = random.sample(image_filenames, num_images)

# Copy the selected images to the output folder
for image_filename in selected_images:
    source_path = os.path.join(folder_path, image_filename)
    destination_path = os.path.join(output_folder_path, image_filename)
    shutil.copyfile(source_path, destination_path)

# Print a confirmation message
print("Selected images have been copied to the output folder.")


Selected images have been copied to the output folder.


In [ ]:
%run example_based.py 

ERROR:root:File `'example_based.py'` not found.


In [ ]:
!pip install innvestigate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 KB 4.2 MB/s eta 0:00:00


usage: ipykernel_launcher.py [-h] [--method METHOD] [--top TOP]
                             [--split SPLIT]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-866c10e5-177e-4b0f-9546-e7f4518b3a5a.json


SystemExit: ignored

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
